In [1]:
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
sys.path.append(project_root)

In [2]:
def print_results(tuner):
    print("Final prompt:", tuner.final_prompt)
    print("Start prompt metric: ", tuner.init_metric)
    print("Final prompt metric: ", tuner.final_metric)

# PromptTuner

Точка входа - ассистент

Он позволяет оптимизировать имеющийся промпт под определенную задачу

Задачу можно задавать датасетом, подаваемым в функцию run, но некоторые оптимизаторы могут работать и без него

In [3]:
from coolprompt.assistant import PromptTuner
tuner = PromptTuner()

2025-06-30 17:01:58,154	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


WARNING 06-30 17:02:00 config.py:1865] Casting torch.bfloat16 to torch.float16.
INFO 06-30 17:02:09 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
INFO 06-30 17:02:09 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='t-tech/T-lite-it-1.0', speculative_config=None, tokenizer='t-tech/T-lite-it-1.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execut

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 06-30 17:02:18 model_runner.py:1077] Loading model weights took 14.2426 GB
INFO 06-30 17:02:24 worker.py:232] Memory profiling results: total_gpu_memory=31.74GiB initial_memory_usage=14.66GiB peak_torch_memory=18.59GiB memory_usage_post_profile=14.69GiB non_torch_memory=0.44GiB kv_cache_size=9.53GiB gpu_memory_utilization=0.90
INFO 06-30 17:02:24 gpu_executor.py:113] # GPU blocks: 11154, # CPU blocks: 4681
INFO 06-30 17:02:24 gpu_executor.py:117] Maximum concurrency for 32768 tokens per request: 5.45x
INFO 06-30 17:02:28 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-30 17:02:28 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
IN

In [5]:
# Запуск без датасета
tuner.run(start_prompt="Write an essay about autumn")

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.43s/it, est. speed input: 174.64 toks/s, output: 39.82 toks/s]


'Write an essay about autumn, focusing on the changes in nature, the emotions it evokes, and the cultural significance of the season. Use descriptive language and provide examples from literature, art, or personal experiences to support your points.'

Ассистент в режиме оптимизации под датасет может решать одну из двух задач: классификацию или генерацию

In [13]:
from datasets import load_dataset

# Классификация
sst2 = load_dataset("stanfordnlp/sst2")
class_dataset = sst2['train']['sentence'][:100]
class_targets = sst2['train']['label'][:100]

tuner.run(
    start_prompt="Classify sentence sentiment",
    task="classification",
    dataset=class_dataset,
    target=class_targets,
    metric="f1"
)

print_results(tuner)

Processed prompts: 100%|██████████| 100/100 [00:06<00:00, 16.66it/s, est. speed input: 5173.62 toks/s, output: 133.31 toks/s]

Final prompt: Please provide a sentence and classify its sentiment as positive, negative, or neutral.
Start prompt metric:  0.6364983164983165
Final prompt metric:  0.8899889988998899


In [6]:
# Генерация
samsum = load_dataset("knkarthick/samsum")
gen_dataset = samsum['train']['dialogue'][:200]
gen_targets = samsum['train']['summary'][:200]

tuner.run(
    start_prompt="Summarize the text",
    task="generation",
    dataset=gen_dataset,
    target=gen_targets,
    metric="meteor"
)

print_results(tuner)

[nltk_data] Downloading package wordnet to /home/sitkina-
[nltk_data]     alena/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/sitkina-
[nltk_data]     alena/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/sitkina-
[nltk_data]     alena/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Processed prompts: 100%|██████████| 200/200 [00:25<00:00,  7.82it/s, est. speed input: 3077.44 toks/s, output: 252.46 toks/s]


Final prompt: Please provide a concise summary of the text, focusing on the main points and key details.
Start prompt metric:  0.25465833610131716
Final prompt metric:  0.3145034036877276


## Language model

При инициализации юзер может подать уже имеющуюся LLM-ку, 

или инициализировать асситента без аргументов, и будет использована дефолтная модель

In [ ]:
from langchain_community.llms import VLLM
# olama + gpt
my_model = VLLM(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    trust_remote_code=True,
    dtype='float16',
)

tuner_with_custom_llm = PromptTuner(model=my_model)
tuner_with_custom_llm.run(start_prompt="Write an essay about autumn")

Также можно изменять конфиг дефолтной модели

In [ ]:
from coolprompt.language_model.llm import DefaultLLM

changed_model = DefaultLLM.init(langchain_config={
    'max_new_tokens': 1000,
    "temperature": 0.0,
})

tuner_with_changed_llm = PromptTuner(model=changed_model)
tuner_with_changed_llm.run(start_prompt="Write an essay about autumn")

## Оптимизатор

Фреймворк поддерживает несколько различных алгоритмов оптимизации промптов

- HyPE
- DistillPrompt
- ReflectivePrompt

Выбор алгоритма производится в параметрах метода run

### HyPE
Мы уже видели его работу до этого - используя заранее заданную системную инструкцию просит модель оптимизировать поданный промпт. оптимизация происходит в одну итерацию

In [14]:
tuner.run(
    start_prompt="Perform Sentiment Classification task.",
    task="classification",
    method="hype",
    dataset=class_dataset,
    target=class_targets,
    metric="accuracy"
)

print_results(tuner)

Processed prompts: 100%|██████████| 100/100 [00:07<00:00, 13.72it/s, est. speed input: 5274.07 toks/s, output: 109.74 toks/s]

Final prompt: Perform Sentiment Classification task by following these steps: 1. Identify the text to be classified. 2. Determine the sentiment categories (e.g., positive, negative, neutral). 3. Analyze the text for sentiment indicators (e.g., positive words, negative words, emoticons). 4. Assign the text to the appropriate sentiment category based on the analysis. 5. Evaluate the accuracy of the classification using a test dataset.
Start prompt metric:  0.93
Final prompt metric:  0.8


### DistillPrompt

Метод автопромптинга, основанный на последовательной дистилляции и агрегации знаний LLM

In [15]:
tuner.run(
    start_prompt="Perform Sentiment Classification task.",
    task='classification',
    dataset=class_dataset,
    target=class_targets,
    method='distill',
    use_cache=True,
    num_epochs=1
)

print_results(tuner)

[2025-06-30 17:25:20,276] - Starting DistillPrompt optimization...
Processed prompts: 100%|██████████| 4/4 [01:44<00:00, 26.18s/it, est. speed input: 10.68 toks/s, output: 77.37 toks/s]


WARNING 06-30 17:43:36 scheduler.py:1481] Sequence group 1905 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


Processed prompts: 100%|██████████| 75/75 [00:07<00:00,  9.57it/s, est. speed input: 1037.94 toks/s, output: 154.09 toks/s]


WARNING 06-30 18:01:58 scheduler.py:1481] Sequence group 2124 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=51


Processed prompts: 100%|██████████| 75/75 [02:52<00:00,  2.30s/it, est. speed input: 58.79 toks/s, output: 629.13 toks/s]
[2025-06-30 18:23:10,480] - Best candidate score in round 0: 0.6440351510774046
[2025-06-30 18:23:10,481] - Best candidate prompt: Classify text sentiment as positive, negative, or neutral using the train dataset to ensure accurate identification.
Processed prompts: 100%|██████████| 25/25 [01:43<00:00,  4.15s/it, est. speed input: 14.07 toks/s, output: 78.94 toks/s]
[2025-06-30 18:24:54,300] - Final best prompt score on validation: 0.5714285714285714
[2025-06-30 18:24:54,301] - Final best prompt: Classify text sentiment as positive, negative, or neutral using the train dataset to ensure accurate identification.
Processed prompts: 100%|██████████| 100/100 [01:54<00:00,  1.14s/it, est. speed input: 53.88 toks/s, output: 217.19 toks/s]

Final prompt: Classify text sentiment as positive, negative, or neutral using the train dataset to ensure accurate identification.
Start prompt metric:  0.3254835996635828
Final prompt metric:  0.5608629721532948


### ReflectivePrompt

Метод автопромптинга на основе эволюционных алгоритмов, использующий подход рефлексивной эволюции для более точного и расширенного поиска оптимальных промптов

In [16]:
tuner.run(
    start_prompt="Perform Sentiment Classification task.",
    task='classification',
    dataset=class_dataset,
    target=class_targets,
    method='reflective',
    problem_description='sentiment classification',
    use_cache=False,
    population_size=4,
    num_epochs=3
)

print_results(tuner)

[2025-06-30 18:32:34,777] - Initializing population...
Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.62s/it, est. speed input: 49.37 toks/s, output: 40.11 toks/s]
[2025-06-30 18:32:36,406] - Evaluating population...


TypeError: Evaluator.evaluate() missing 1 required positional argument: 'template'

## Evaluator

Если вы укажете датасет для оптимизации, фреймворк позволяет измерять метрики, получаемые при использовании определенного промпта

На данный момент поддерживаются
- accuracy и f1 для классификации
- meteor, rouge и bleu для генерации

Метрики находятся в публичных полях ассистента

In [ ]:
print("Start prompt metric: ", tuner.init_metric)
print("Final prompt metric: ", tuner.final_metric)